In [5]:
import os
import glob
from pathlib import Path

import joblib
from joblib import load

import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

SAMPLE_RATE = 44100
HOP_LENGTH = 256         
N_FFT = 1024          
N_MFCC = 13                 

def extract_mfcc(y) -> np.ndarray:
    
    y, _ = librosa.load(y, sr=SAMPLE_RATE)
    
    mfcc = librosa.feature.mfcc(
        y=y,
        sr=SAMPLE_RATE,
        n_mfcc=N_MFCC,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        center=True)
    delta = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    feats = np.vstack([mfcc, delta, delta2])
    return aggregate_stats(feats)

def aggregate_stats(feats: np.ndarray) -> np.ndarray:
    out = []
    for row in feats:
        vals = np.asarray(row, dtype=np.float32)
        out.extend([
            np.mean(vals),
            np.std(vals),
            np.median(vals),
            np.max(vals) - np.min(vals)
        ])
    return np.asarray(out, dtype=np.float32)


In [6]:
from sklearn.model_selection import train_test_split

in_path_music = os.path.abspath("mixed_up_data_speak_segmented/speak")

speak_files = [f for f in os.listdir(in_path_music)]

df_speak = pd.DataFrame({
    "mfcc_coeff": [extract_mfcc(os.path.join(in_path_music, f)) for f in tqdm(speak_files, desc="Estrazione MFCC da speak_files")],
    "label":      1
})

Estrazione MFCC da speak_files:   0%|          | 0/2838 [00:00<?, ?it/s]

In [7]:
in_path_noise = os.path.abspath("mixed_up_data_speak_segmented/no_speak")
no_speak_files = [f for f in os.listdir(in_path_noise)]

df_no_speak = pd.DataFrame({
    "mfcc_coeff": [extract_mfcc(os.path.join(in_path_noise, f)) for f in tqdm(no_speak_files, desc="Estrazione MFCC da no_speak_files")],
    "label":      0
})

train = pd.concat([df_speak, df_no_speak], ignore_index=True)

Estrazione MFCC da no_speak_files:   0%|          | 0/2620 [00:00<?, ?it/s]

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

model = LogisticRegression(C=1.0, penalty='l2', solver='liblinear', max_iter=1000)

X = np.vstack(train["mfcc_coeff"].values) 
y = train["label"].values                  

X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

model.fit(X_train, y_train)
print("Fine dell'addestramento")

Fine dell'addestramento


In [9]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85       524
           1       0.86      0.87      0.86       568

    accuracy                           0.86      1092
   macro avg       0.86      0.86      0.86      1092
weighted avg       0.86      0.86      0.86      1092



In [10]:
def predict_noisy_probability(wav_path):
    emb = extract_mfcc(wav_path).reshape(1, -1)
    emb = scaler.transform(emb)
    probs = model.predict_proba(emb)[0]   # array di lunghezza 2
    return probs

test_files = [
    "audio_test/music_pure.wav",
    "audio_test/noise_pure.wav",
    "audio_test/voice_base_music.wav",
    "audio_test/voice_base_pure.wav",
    "audio_test/voice_base_noise.wav"
]
for test_file in test_files:
    if os.path.exists(test_file):
        p_no_speak, p_speak = predict_noisy_probability(test_file)
        print(f"{test_file} → no_speak: {p_no_speak:.3f}, speak: {p_speak:.3f}")
    else:
        print(f"File di test non trovato: {test_file}")

audio_test/music_pure.wav → no_speak: 0.020, speak: 0.980
audio_test/noise_pure.wav → no_speak: 0.867, speak: 0.133
audio_test/voice_base_music.wav → no_speak: 0.001, speak: 0.999
audio_test/voice_base_pure.wav → no_speak: 0.000, speak: 1.000
audio_test/voice_base_noise.wav → no_speak: 0.000, speak: 1.000


# Breve Descrizione
Il modello è stato allenato su più dati rispetto alle alternative precedenti, anche perché si era ragionevolmente sicuri funzioansse. Gli MFCC non sono assolutamente adatti a riconoscere sottofondo musicale da noise generico ma sono adatti a distinguere speak/no speak: dimostrato da un'ottima accuracy all'85%, la componente di errore è data principalmente dal fatto che ci potrebbero essere audio music only dove questa, oltre che avere le componenti armoniche alle stesse quefrency udibili in scala Mel del parlato, magari varia tanto rapidamente quanto la voce stessa ed è, quindi, robusta alle first e second derivatives degli MFCC, o meglio, ha lo stesso "identikit" della voce. Il modello è allenato su finestre di 3 secondi ed è sempre ragionevolmente sicuro che ci sia voce o meno, come si vede dalla stampa di sopra: l'errore che commette sulla musica pura, viene mitigato segmentando l'audio e prendendo una decisione sulla majority, come si può apprezzare sotto. Questo è interessante, in quanto l'audio preso in considerazione in music_pure ha componente di tono ad alta variazione (assolo di chitarra) solo nella parte finale dell'instrumental, di conseguenza l'armonica a basse frequenza e costante per gran parte della canzone, il che avrebbe "identikit" soprattutto di First e Second Derivatives molto diverso dal parlato, viene 'diluito' dalle componenti finali che 'innalzano' Standard Error, Mean... e le altre metriche utilizzate. Ragionando per Majority sulle finestre da 3s, oltre che poter far riferimento a formati su cui sia stato effettivamente allenato il modello, permette di mitigare problemi di questo tipo, anche se, magari, audio con poco parlato, ma pur sempre presente, e molto silenzio/musica vengono etichettati come no_speak, in realtà questo sarebbe anche accettabile per l'utilizzo che vogliamo fare di questo classificatore insomma e perfettamente coerente con la soglia di 85% ottenuta su test set

In [11]:
MODEL_OUT = "classifier_mfcc_speak.joblib"

print(f"Modello Logistic Regression salvato in {MODEL_OUT} …")
joblib.dump({'scaler': scaler, 'model': model}, MODEL_OUT)

Modello Logistic Regression salvato in classifier_mfcc_speak.joblib …


['classifier_mfcc_speak.joblib']

In [15]:
import numpy as np
import librosa
from sklearn.linear_model import LogisticRegression
from collections import Counter
from typing import List

WINDOW_SECONDS = 3

def aggregate_stats(feats: np.ndarray) -> np.ndarray:

    out = []
    for row in feats:
        vals = row.astype(np.float32)
        out.extend([
            np.mean(vals),
            np.std(vals),
            np.median(vals),
            np.max(vals) - np.min(vals)
        ])
    return np.asarray(out, dtype=np.float32)

def extract_mfcc_from_signal(y: np.ndarray) -> np.ndarray:
   
    mfcc    = librosa.feature.mfcc(y=y, sr=SAMPLE_RATE,
                                   n_mfcc=N_MFCC,
                                   n_fft=N_FFT,
                                   hop_length=HOP_LENGTH,
                                   center=True)
    delta1  = librosa.feature.delta(mfcc, order=1)
    delta2  = librosa.feature.delta(mfcc, order=2)
    feats   = np.vstack([mfcc, delta1, delta2])
    return aggregate_stats(feats)

def segment_audio(path: str, window_sec: float = WINDOW_SECONDS) -> List[np.ndarray]:

    y, _ = librosa.load(path, sr=SAMPLE_RATE)
    win_len = int(window_sec * SAMPLE_RATE)
    n_segs  = int(np.ceil(len(y) / win_len))
    segments = []
    for i in range(n_segs):
        start = i * win_len
        end   = start + win_len
        seg   = y[start:end]
        if len(seg) < win_len:
            seg = np.pad(seg, (0, win_len - len(seg)), mode='constant')
        segments.append(seg)
    return segments

def extract_features_per_segment(path: str) -> np.ndarray:

    segments = segment_audio(path)
    feats = [extract_mfcc_from_signal(seg) for seg in segments]
    return np.vstack(feats)

def classify_segments(path: str, model: LogisticRegression, scaler: StandardScaler) -> List[int]:
  
    X = extract_features_per_segment(path)
    X_scaled = scaler.transform(X)
        
    return model.predict(X_scaled).tolist()

def majority_vote(preds: List[int]) -> int:
   
    cnt = Counter(preds)
    return cnt.most_common(1)[0][0]

def global_decision_majority(path: str, model: LogisticRegression, scaler: StandardScaler) -> int:

    seg_preds = classify_segments(path, model, scaler)
    return majority_vote(seg_preds)

if __name__ == "__main__":

    # questa logica sarà da modificare su ComfyUI per raccogliere input da IO
    test_files = [
    "audio_test/music_pure.wav",
    "audio_test/noise_pure.wav",
    "audio_test/voice_base_music.wav",
    "audio_test/voice_base_pure.wav",
    "audio_test/voice_base_noise.wav"
    ]
    
    data = load(MODEL_OUT)
    scaler = data['scaler']
    model  = data['model']

    for audio_file in test_files:
        segment_preds = classify_segments(audio_file, model, scaler)

        base, _ = os.path.splitext(audio_file)
        global_pred = global_decision_majority(audio_file, model, scaler)
        print(f"\nDecisione per {base} (majority vote): {global_pred}")


Decisione per audio_test/music_pure (majority vote): 0

Decisione per audio_test/noise_pure (majority vote): 0

Decisione per audio_test/voice_base_music (majority vote): 1

Decisione per audio_test/voice_base_pure (majority vote): 1

Decisione per audio_test/voice_base_noise (majority vote): 1
